In [ ]:
import pathlib

import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt

import hands_on_machine_learning as homl

datasets_dir = pathlib.Path('datasets')

housing_dir = datasets_dir / 'housing'
housing_csv = housing_dir / 'housing.csv'

if not housing_csv.is_file():
    homl.datasets.fetch.housing(housing_dir)

housing_data = pd.read_csv(housing_csv)

### Take a quick look at the data structure

The goal of the project is to try to predict the sale value of homes in regions where the existing data on home values is sparse.  So, we will try to analyze home-buyers' behavior in terms of other data such as income, population density, and various facts about the houses themselves.

In [ ]:
housing_data.head()

In [ ]:
housing_data.info()

In [ ]:
housing_data['ocean_proximity'].value_counts()

In [ ]:
housing_data.describe()

In [ ]:
housing_data.hist(bins=50, figsize=(20, 15))
plt.show()

### Create a Test Set

First create a naive test set by simply reserving 20% of the data, chosen randomly.

In [ ]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(housing_data, test_size=0.2, random_state=42)

In [ ]:
len(train_set), len(test_set)

In [ ]:
housing_data['income_cat'] = pd.cut(
    housing_data['median_income'],
    bins=[0., 1.5, 3.0, 4.5, 6., np.inf],
    labels=[1, 2, 3, 4, 5]
)
housing_data['income_cat'].hist()

However, since income levels probably have a significant effect on people's home-buying behavior, the test set will be more representative if we use a *stratified* sample, making sure to take a good number of samples from each income bracket.

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing_data, housing_data['income_cat']):
    strat_train_set = housing_data.loc[train_index]
    strat_test_set = housing_data.loc[test_index]

for strat_set in (strat_train_set, strat_test_set):
    strat_set.drop('income_cat', axis=1, inplace=True)

## Discover and visualize the data to gain insights

We will create various plots to help us understand what we might want to look for.

In [ ]:
housing_data = strat_train_set.copy()
housing_data.plot(kind='scatter', x='longitude', y='latitude')

In [ ]:
housing_data.plot(kind='scatter', x='longitude', y='latitude', alpha=0.1)

This one shows that the house value is strongly dependent on location, with expensive homes concentrated in the Bay Area and LA/San Diego.

In [ ]:
housing_data.plot(kind='scatter', x='longitude', y='latitude', alpha=0.4,
    s=housing_data['population']/100, label='population', figsize=(10,7),
    c='median_house_value', cmap=plt.get_cmap('jet'), colorbar=True
)
plt.legend()

### Looking for correlations

We will see if any of the attributes in the data set appear to be related to one another.

In [ ]:
corr_matrix = housing_data.corr()

In [ ]:
corr_matrix['median_house_value'].sort_values(ascending=False)

In [ ]:
attributes = ['median_house_value', 'median_income', 'total_rooms', 'housing_median_age']
pd.plotting.scatter_matrix(housing_data[attributes], figsize=(12, 8));

### Experimenting with attribute combinations

It may be useful to define some derived attributes, which may have better properties for machine learning, or may have more relation to what we expect should affect home prices.

In [ ]:
housing_data['rooms_per_household'] = housing_data['total_rooms'] / housing_data['households']
housing_data['bedrooms_per_room'] = housing_data['total_bedrooms'] / housing_data['total_rooms']
housing_data['population_per_household'] = housing_data['population'] / housing_data['households']

In [ ]:
corr_matrix = housing_data.corr()
corr_matrix['median_house_value'].sort_values(ascending=False)

## Prepare the data for machine learning algorithms

First split the labels from the feature data. We will apply various transformations to the feature data to clean it.

In [ ]:
housing_features = strat_train_set.drop('median_house_value', axis=1)
housing_labels = strat_train_set['median_house_value'].copy()

### Data cleaning

In [ ]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='median')

In [ ]:
housing_features_numerical = housing_features.drop('ocean_proximity', axis=1)

In [ ]:
imputer.fit(housing_features_numerical);

In [ ]:
imputer.statistics_

In [ ]:
X = imputer.transform(housing_features_numerical)
housing_features_imputed = pd.DataFrame(X, columns=housing_features_numerical.columns, index=housing_features_numerical.index)

### Handling text and categorical attributes

In [ ]:
housing_cat = housing_data[['ocean_proximity']]
housing_cat.head(10)

The `'ocean_proximity'` field only takes on a limited set of values.  It is a bit inconvenient to deal with strings, so we convert them to numbers.  One possibility is `OrdinalEncoder`, which gives them numbers like an `enum`.

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

ordinal_encoder = OrdinalEncoder()
housing_cat_encoded = ordinal_encoder.fit_transform(housing_cat)
housing_cat_encoded[:10]

However, these categories are not really *ordinal* in nature, so it makes more sense to use a one-hot encoding.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

cat_encoder = OneHotEncoder()
housing_cat_onehot = cat_encoder.fit_transform(housing_cat)
housing_cat_onehot.toarray()

In [ ]:
cat_encoder.categories_

### Custom transformers

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room=True):
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
        population_per_household = X[:, population_ix] / X[:, households_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

### Transformation pipelines

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

numerical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('attributes_adder', CombinedAttributesAdder()),
    ('std_scaler', StandardScaler())
])

In [ ]:
from sklearn.compose import ColumnTransformer

numerical_attributes = list(housing_features_numerical)
category_attributes = ['ocean_proximity']

full_pipeline = ColumnTransformer([
    ('num', numerical_pipeline, numerical_attributes),
    ('cat', OneHotEncoder(), category_attributes)
])

In [ ]:
housing_prepared = full_pipeline.fit_transform(housing_data)

## Select and train a model

### Training and evaluating on the training set

In [ ]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(housing_prepared, housing_labels)

In [ ]:
some_data = housing_data.iloc[:5]
some_labels = housing_labels.iloc[:5]
some_data_prepared = full_pipeline.transform(some_data)
print("Predictions:", lin_reg.predict(some_data_prepared))
print("Labels:", list(some_labels))

In [ ]:
from sklearn.metrics import mean_squared_error

housing_predictions = lin_reg.predict(housing_prepared)
lin_mse = mean_squared_error(housing_labels, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor()
tree_reg.fit(housing_prepared, housing_labels)

In [ ]:
housing_predictions = tree_reg.predict(housing_prepared)
tree_mse = mean_squared_error(housing_labels, housing_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

### Better evaluation using cross-validation

In [ ]:
from sklearn.model_selection import cross_val_score

def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

In [ ]:
scores = cross_val_score(tree_reg, housing_prepared, housing_labels,
    scoring='neg_mean_squared_error', cv=10)
tree_rsme_scores = np.sqrt(-scores)

display_scores(tree_rsme_scores)

In [ ]:
scores = cross_val_score(lin_reg, housing_prepared, housing_labels,
    scoring='neg_mean_squared_error', cv=10)
lin_rsme_scores = np.sqrt(-scores)

display_scores(lin_rsme_scores)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor()
forest_reg.fit(housing_prepared, housing_labels)

housing_predictions = forest_reg.predict(housing_prepared)
forest_mse = mean_squared_error(housing_labels, housing_predictions)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

In [ ]:
scores = cross_val_score(forest_reg, housing_prepared, housing_labels,
    scoring='neg_mean_squared_error', cv=10)
forest_rsme_scores = np.sqrt(-scores)

display_scores(forest_rsme_scores)

## Fine-tune your model

### Grid search

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    {'n_estimators': [3, 10, 30, 100], 'max_features': [2, 4, 6, 8, 10, 12]},
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]}
]

forest_reg = RandomForestRegressor()

grid_search = GridSearchCV(
    forest_reg, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True
)

grid_search.fit(housing_prepared, housing_labels)

In [ ]:
grid_search.best_params_

In [ ]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres['mean_test_score'], cvres['params']):
    print(np.sqrt(-mean_score), params)

### Evaluate your system on the test set

In [ ]:
final_model = grid_search.best_estimator_

X_test = strat_test_set.drop('median_house_value', axis=1)
y_test = strat_test_set['median_house_value'].copy()

X_test_prepared = full_pipeline.transform(X_test)

final_predictions = final_model.predict(X_test_prepared)

final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)
final_rmse

In [ ]:
from scipy import stats

confidence = 0.95
squared_errors = (final_predictions - y_test) ** 2
np.sqrt(stats.t.interval(confidence, len(squared_errors) - 1, loc=squared_errors.mean(), scale=stats.sem(squared_errors)))